# Archive statistics

In [1]:
import ipywidgets as widgets
widgets.Widget.widget_types
import requests
import geopy
#import pandas as pd
from IPython.display import display

%matplotlib inline

#import seaborn as sns
#import plotly
from ipywidgets import interact
from ipywidgets import Label, Layout

from geopy.geocoders import Nominatim
geolocator = Nominatim(timeout=3)

from IPython.display import clear_output

#import json
#import datacube

#dc = datacube.Datacube(app='dc-example')

import geopandas
import itertools

import os, json, requests, time
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import gzip, zipfile

import datacube
from datacube.utils import geometry
from datacube.model import Range
from datetime import datetime
from datetime import timedelta
import shapely
from datacube.api import GridWorkflow
from datacube.utils.geometry import CRS
from shapely.geometry import mapping

import json

dc = datacube.Datacube()

import matplotlib as mpl
import matplotlib.cm
import matplotlib.colors
import numpy as np

In [2]:
products = dc.list_products()['description'].tolist()
products_name = dc.list_products()['name'].tolist()
center = [-26, 132]
zoom = 3

In [3]:
# Create a dict to allow product picker with description
products_name = dc.list_products()['name'].tolist()
product_dict = {}
for idx, val in enumerate(products):
    product_dict[val]= products_name[idx]

In [4]:
from __future__ import print_function

from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

from traitlets import link

In [5]:
aoi_picker = widgets.Text(value='Canberra',placeholder='update this field',
    description='placename search',
    disabled=False)
def handle_submit(sender):
    global location
    global center
    global zoom
    location = geolocator.geocode(aoi_picker.value)
    center = [location.latitude, location.longitude]
    zoom = 13
    print(center)
aoi_picker.on_submit(handle_submit)

In [6]:
#TODO Replace with datapicker widget when ipywidgets devs sort this out
from_date_picker = widgets.Text(value='2000-12-31',placeholder='update this field',
    description='start date',
    disabled=False)
def handle_from_date(sender):
    print(from_date_picker.value)
from_date_picker.on_submit(handle_from_date)

In [7]:
#TODO Replace with datapicker widget when ipywidgets devs sort this out
to_date_picker = widgets.Text(value='2001-01-01',placeholder='update this field',
    description='end date',
    disabled=False)
def handle_to_date(sender):
    print(to_date_picker.value)
to_date_picker.on_submit(handle_to_date)

In [8]:
product_picker = widgets.RadioButtons(
    options=products,
    description='product:',
    disabled=False
)

In [9]:
zoom_picker =widgets.RadioButtons(
    options=['city','regional', 'country'],
    description='zoom:',
    disabled=False
)

In [10]:
#stats_picker =widgets.RadioButtons(
#    options=['count files with bounds intersecting AOI','count all files'],
#    description='zoom:',
#    disabled=False
#)

In [14]:
accordion = widgets.Accordion(children=[product_picker,from_date_picker, to_date_picker, aoi_picker,zoom_picker])#,stats_picker ])
accordion.set_title(0, 'product')
accordion.set_title(1, 'from date')
accordion.set_title(2, 'to date')
accordion.set_title(3, 'placename')
accordion.set_title(4, 'zoom level')
#accordion.set_title(5, 'statistic')
accordion

In [15]:
def geojson_grid_count(product_name, query):
    #from_time = 
    #to_time
    dc = datacube.Datacube()
    gw = datacube.api.GridWorkflow(dc.index, product=product_name)

    cells = gw.list_cells(group_by='solar_day', product=product_name, **query)#time=(from_time, to_time))

    geojson = cells_list_to_featurecollection(cells)

    return(geojson)

def cells_list_to_featurecollection(cells_dict):
    return {'type': 'FeatureCollection',
            'features': [tile_to_geojsonfeature(tile, cell=cell_to_str(cell))
                         for cell, tile in cells_dict.items()]}


def tile_to_geojsonfeature(tile, **props):
    geometry = tile.geobox.extent.to_crs(CRS('EPSG:4326')).__geo_interface__
    feature = {'type': 'Feature',
               'geometry': geometry,
               'properties': dict(count=str(len(tile.sources)),
                                  **props)}
    return feature


def cell_to_str(cell):
    return ', '.join(str(el) for el in cell)


In [16]:
label = Label(layout=Layout(width='100%'))
def hover_handler(event=None, id=None, properties=None):
            
            label.value = properties['count']

In [17]:
def n_colors(n, colormap=mpl.cm.hot):
    data = np.linspace(0.0,1.0,n)
    c = [mpl.colors.rgb2hex(d[0:3]) for d in colormap(data)]
    return c

In [18]:
#drawcontrol = DrawControl(polygon={'shapeOptions': {'color': '#0000FF'}}, polyline={'shapeOptions': {'color': '#00FF00'}},
#                      circle={'shapeOptions': {'color': '#0000FF'}}, rectangle={'shapeOptions': {'color': '#0000FF'}})
# Could include all drawcontrols but dc needs a simple geom?
drawcontrol = DrawControl(rectangle={'shapeOptions': {'color': '#0000FF'}})

In [40]:
update_map = widgets.Button(description="Update map on click")
m = None
drawcontrol.last_draw['geometry'] = None
def update_map_clicked(button):
    global m
    global zoom
    global drawcontrol
    global data
    
    # clear the map
    if m:
        m.close()
    # collect and set the zoom level
    if (zoom_picker.value is 'city'):

        zoom = 13
    if (zoom_picker.value is 'regional'):
        zoom = 7
    else:
        zoom = 3
        
    # add the DrawControl to the map
    m = Map(center=center, zoom=zoom)

    m.add_control(drawcontrol)

    product = product_dict[product_picker.value]
    
    
    if drawcontrol.last_draw['geometry'] is not None:
        list_lat = []
        list_lon = []
        for i in drawcontrol.last_draw['geometry']['coordinates'][0]:
            list_lon.append(i[0])
            list_lat.append(i[1])

        expression = {'time': (from_date_picker.value, to_date_picker.value),
            'lat': (min(list_lat), max(list_lat)),
            'lon': (min(list_lon), max(list_lon)),
            }
        print('Processing:', product, expression)
        
    else:
        expression = {'time': (from_date_picker.value, to_date_picker.value)}
    
        print('Processing:', product, from_date_picker.value, to_date_picker.value)
    
    data = geojson_grid_count(product, expression)
    
    counts = []
    for property in data['features']:
        counts.append(property['properties']['count'])

    n_features = max(set(list(map(int, counts)))) - min(set(list(map(int, counts))))
    
    if n_features == 0:
        n_features = 1

    colors = n_colors(n_features)
    
    for feature in data['features']:
        feature['properties']['style'] = {'color':colors[int(feature['properties']['count'])- min(set(list(map(int, counts))))-1], 'weight': 1, 'fillColor':colors[int(feature['properties']['count'])- min(set(list(map(int, counts)))) -1], 'fillOpacity':0.5}
            
            
    layer = GeoJSON(data=data, hover_style={'fillColor': 'red'})
        
    layer.on_hover(hover_handler)
    m.add_layer(layer)
    m.center = center
    display(m, label)
    
display(update_map)
update_map.on_click(update_map_clicked)

Processing: ls7_pq_albers 2000-12-31 2001-01-01


In [55]:
save_csv = widgets.Text(value='file.csv',placeholder='enter your destination file path',
    description='CSV export',
    disabled=False)
def handle_save(sender):
    geopandas = gpd.GeoDataFrame.from_features(data['features'])
    geopandas.to_csv('file.csv')
    print('file saved')

save_csv.on_submit(handle_save)

In [56]:
accordion0 = widgets.Accordion(children=[save_csv])
accordion0.set_title(0, 'Export CSV')
accordion0

file saved


In [ ]:
# initialise the DrawControl to capture the geometry
import cesiumpy
import geopandas as gpd

update_cesium = widgets.Button(description="Update globe on click")
options = dict(animation=True, baseLayerPicker=False, fullscreenButton=False,
                geocoder=False, homeButton=False, infoBox=False, sceneModePicker=True,
                selectionIndicator=False, navigationHelpButton=False,
                timeline=False, navigationInstructionsInitiallyVisible=False)
v = None
def update_cesium_clicked(button):
    v = cesiumpy.Viewer(**options)
    v.entities.add(cesiumpy.Wall(positions=g, maximumHeights=10e5, minimumHeights=0, material=cesiumpy.color.RED))
    v
    layer = GeoJSON(data=data, hover_style={'fillColor': 'red'})
    



df = gpd.read_file(fname)

print(type(df))

df
    
    
display(update_cesium)
update_cesium.on_click(update_cesium_clicked)

In [ ]:
button = widgets.Button(description="Generate Report")
def on_button_clicked(button):
    print('Generating your report now, hold onto your shorts')

    #print(product_picker.value, sensor_picker.value, from_date_picker.value, to_date_picker.value, dc.last_draw['geometry'])
    
    if (stats_picker.value == 'count all files'):
        product = product_dict[product_picker.value]
        results = grid_file_count(product, from_date_picker.value, to_date_picker.value )
        
        dc = datacube.Datacube()
        gw = datacube.api.GridWorkflow(dc.index, product=product)
        print(product, from_date_picker.value, to_date_picker.value)
        cells = gw.list_cells(product=product, time=(from_date_picker.value, to_date_picker.value))
        
        features = []
        for idx, data in cells.items():
            print(idx, data.shape[0], data.geobox.extent.to_crs(geometry.CRS('EPSG:4326')).__geo_interface__['coordinates'])
            
            features.append({'id' : idx, 'count': data.shape[0], 'geometry' : \
                            mapping(shapely.geometry.polygon.Polygon(data.geobox.extent.to_crs(datacube.utils.geometry.CRS('EPSG:4326')).__geo_interface__['coordinates'])),\
                           'color': '#%02x%02x%02x' % (rgb_combo( rgb_range, max_value,results[key].shape[0])),\
                           'type': 'Feature'})
    else:
        print('No statistic selected')
    
display(button)
button.on_click(on_button_clicked)

In [ ]:
# Example of tabs - ignore for now
#list = ['P0', 'P1', 'P2', 'P3', 'P4']
#children = [accordion,update_map]
#print(children)
#tab = widgets.Tab(children=children)
#tab

ul_lon, ul_lat = 77.83, 17.84 # upper left longitude, latitude
lr_lon, lr_lat = 78.00, 17.67 # lower right longitude, latitude
date_start, date_end = from_date_picker.value, to_date_picker.value  # start date and end date for time range selection  
sensor_list=["olitirs8", "tm5", "etm7"] # list of sensors to use

polygon_list = [[ul_lon, ul_lat], [lr_lon, ul_lat],[lr_lon, lr_lat],[ul_lon, lr_lat],[ul_lon, ul_lat]]

wrs_query = 'http://api.remotepixel.ca/wrs2tiles?search=poly:'+str(polygon_list)

post_query = requests.get(wrs_query)
wrs_search_result = json.loads(post_query.text)

path_row = []
for item in wrs_search_result['results']:
    path_row.append(str(item['path'])+"_"+str(item['row']))
path_row

tips = sns.load_dataset('tips')